In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display # to display plots on jypter notebooks

## Eigenvalues and Eigenvectors of Matrices

In [ ]:
def eigen(matrix):
    # Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(matrix)

    # Output results
    print("Matrix:")
    print(matrix)

    # Display eigenvectors alongside their eigenvalues
    print("\nEigenvalue-Eigenvector pairs:")
    for i in range(len(eigenvalues)):
        print(f"λ = {eigenvalues[i]:.4f}, v = any multiple of {np.round(eigenvectors[:, i],4)}")

In [ ]:
# Let's look at three different examples of 2x2 matrices

matrix1 = np.array([
    [-1, 2],
    [ 0, 1]
])

matrix2 = np.array([
    [1, 1.1],
    [0, 1]
])

matrix3 = np.array([
    [ 1, 5],
    [ 3, 1]
])

print("Example 1")
eigen(matrix1)
print()
print("Example 2")
eigen(matrix2)
print()
print("Example 3")
eigen(matrix3)


## Linear Transformations

Huge thank you to Dakota Blanchard for creating the animations!

In [ ]:
animations = [] # Store animations

### MAIN FUNCTION ###
def transform(ascii_art, matrix):
    # Convert ASCII art to coordinates
    original_coords = _ascii_art_to_coords(ascii_art)
    
    # Apply the linear transformation
    transformed_coords = _apply_linear_transform(original_coords, matrix)
    
    # Create an animation of the linear transformation
    anim = _animate_graph(original_coords, transformed_coords, matrix)
    
    return anim

In [ ]:
### HELPER FUNCTIONS ###
def _ascii_art_to_coords(ascii_art):
    """Convert ascii art into a list of coordinates for each character"""
    coords = [] #  Initialize coordinates of ascii characters
    max_y = len(ascii_art) - 1 #  Store height of ascii art
    for y_idx, line in enumerate(ascii_art): #  loop thru each row of text
        y = max_y - y_idx #  to imitate a real graph, the top of ascii art will be the highest y value
        for x, char in enumerate(line): #  loop thru each character in row
            if char != ' ': #  if not a space, add (x, y, char) tuple to list
                coords.append((x, y, char))
    return coords #  return list of (x, y, char) tuples
                
def _apply_linear_transform(coords, matrix):
    """Apply a linear transformation to each point using a given matrix"""
    new_coords = [] 
    for x, y, char in coords: #  loop thru each point in coords list
        vector = np.array([x, y]) #  create a vector from coordinates
        new_pos = matrix @ vector #  @ operator used for matrix multiplication
        new_coords.append((new_pos[0], new_pos[1], char)) #  add transformed coords to list, return them
    return new_coords

def _animate_graph(original, transformed, matrix=None):
    """Create an interactive animation of the linear transformation"""
    # Extract both the original and transformed coordinates
    ox = np.array([x for x, y, c in original])
    oy = np.array([y for x, y, c in original])
    tx = np.array([x for x, y, c in transformed])
    ty = np.array([y for x, y, c in transformed])

    # Setup scatter plot
    # Plot original points as blue squares
    fig, ax = plt.subplots(figsize=(8, 6))
    scat = ax.scatter(ox, oy, c='blue', marker='s')
    
    # Make a single array of both original+transformed points
    all_x = np.concatenate([ox, tx])
    all_y = np.concatenate([oy, ty])
    # Use array to set x,y limits to fit points into scatter plot
    ax.set_xlim(all_x.min() - 1, all_x.max() + 1)
    ax.set_ylim(all_y.min() - 1, all_y.max() + 1)
    ax.set_aspect('equal') # 1:1 aspect ratio
    
    # Add a grid & title
    ax.grid(True)
    ax.set_title("Linear Transformation of ASCII Art")

    # Compute eigenvectors
    if matrix is not None: # If the matrix exists
        eigvals, eigvecs = np.linalg.eig(matrix) # compute eigenvalues (how much the matrix stretches/shrinks a direction) and eigenvectors
        eigvecs = eigvecs / np.linalg.norm(eigvecs, axis=0) # normalize eigenvector
        quiv = ax.quiver( 
            [0]*len(eigvals), [0]*len(eigvals), #  arrows start at (0,0)
            eigvecs[0,:], eigvecs[1,:], #  x & y component of each eigenvector
            angles='xy', scale_units='xy', scale=1, color='red' #  draw eigenvectors as red arrows
        )
        
        # Add eigenvalue labels near each arrow
        labels = []
        for i in range(len(eigvals)):
            label = ax.text(eigvecs[0,i]*3, eigvecs[1,i]*3, f"λ={eigvals[i]:.2f}", color='red')
            labels.append(label)

    frames = 60 # how smooth the animation is

    def update(frame):
        """Updates the scatterplot once per frame"""
        t = frame / (frames - 1) #  time of animation progression from 0-1
        # Interpolate all points inbetween
        x = (1-t) * ox + t * tx # interpolate x, y coords over time
        y = (1-t) * oy + t * ty
        scat.set_offsets(np.c_[x, y]) # stack x, y into 2 column array for matplotlib to redraw points over time
        scat.set_color(plt.cm.viridis(t)) # change color over time

        if matrix is not None:
            # Scale eigenvectors by eigenvalues
            U = eigvecs * ((1-t) + t * eigvals)  # linearly interpolate eigenvector arrows
            quiv.set_UVC(U[0,:], U[1,:]) # animate arrow scale
        return (scat, quiv)

    # Create the animation
    anim = FuncAnimation(fig, update, frames=frames, interval=50, blit=False, repeat=False)
    
    plt.close(fig) # prevent duplicates
    
    return anim

In [ ]:
# Let's start with a picture to transform
ascii_art = [
    "     *     ",
    "    ***    ",
    "   *****   ",
    "  *******  ",
    " ********* ",
    "    ***    ",
    "    ***    "
]

In [ ]:
# Example 1 (using matrix1)
anim1 = transform(ascii_art, matrix1)
display(HTML(anim1.to_jshtml()))

In [ ]:
# Example 2 (using matrix2)
anim2 = transform(ascii_art, matrix2)
display(HTML(anim2.to_jshtml()))

In [ ]:
# Example 3 (using matrix3)
anim3 = transform(ascii_art, matrix3)
display(HTML(anim1.to_jshtml()))

### Bonus Materials

In [ ]:
import math
# Sample Linear Transformations you can do

# Rotation (counterclockwise)
theta_deg = 45
theta = math.radians(theta_deg)
rotation_matrix = np.array([
    [math.cos(theta), -math.sin(theta)],
    [math.sin(theta),  math.cos(theta)]
])

# Scaling
scaling_matrix = np.array([
    [2, 0],
    [0, 1]
])

# Shearing (x-direction)
shear_matrix = np.array([
    [1, 0.5],
    [0, 1]
])